In [ ]:
# Importar as bibliotecas

import sys
import json
import os
import pandas

In [ ]:
# Trazer as opções de configuração do JSON

json_file = os.path.abspath('../options.json')

with open(json_file, 'r', encoding='utf-8') as j_file:
    json_data = json.load(j_file)

In [ ]:
# Lista os investimentos escolhidos

choosed_investments = json_data["INVESTMENTS"]
choosed_investments = list(choosed_investments.keys())

In [ ]:
# Lista o histórico dos investimentos escolhidos

data_dir = os.path.join(os.path.abspath('../archives/'), json_data['CONFIG']['DIR_CVM_CSV_NAME'])

list_csv_archive = os.listdir(data_dir)
list_csv_archive_filtered = []

if len(list_csv_archive) == 0:
    print('Não há arquivos na pasta "./app/archives/"' + json_data['CONFIG']['DIR_CVM_CSV_NAME'])
    sys.exit()

for csv_archive_month in list_csv_archive: 

    list_investments_month = pandas.read_csv(os.path.join(data_dir, csv_archive_month), sep=';', encoding='utf-8')
    list_investments_month_filtered = list_investments_month.query('CNPJ_FUNDO_CLASSE in @choosed_investments')
    
    list_csv_archive_filtered.append(list_investments_month_filtered)

if len(list_csv_archive_filtered) == 0:
    print('Não há nenhum dado populado')
    sys.exit()

list_investments_filtered = pandas.concat(list_csv_archive_filtered, ignore_index=True)

In [ ]:
# Faz a ordenação dos investimentos por CNPJ e data

list_investments_filtered = list_investments_filtered.sort_values(by=["CNPJ_FUNDO_CLASSE", "DT_COMPTC"])

In [ ]:
# Reseta o índice da coluna de identificação

list_investments_filtered = list_investments_filtered.reset_index()

In [ ]:
# Criação da coluna do nome dos fundos

def search_name(element):
    return json_data['INVESTMENTS'][element]['Fundo']

list_investments_filtered['NOME_FUNDOS'] = list_investments_filtered.CNPJ_FUNDO_CLASSE.map(search_name)

In [ ]:
# Criação da coluna da valorização de cotas

started_value_cota = 0
preview_cnpj = ''

def string_to_number(element):
    return int(element)

def is_first_date_previous_than_second (investment_date_start, investment_date_row):
    start_year, start_month, start_day = map(string_to_number, str(investment_date_start).split('-'))
    row_year, row_month, row_day = map(string_to_number, str(investment_date_row).split('-'))

    if start_year > row_year:
        return False
    elif start_year < row_year:
        return True

    if start_month > row_month:
        return False
    elif start_month < row_month:
        return True
    
    if start_day > row_day:
        return False
    elif start_day < row_day:
        return True
   
    return False

def calculate_valuation(row):
    global started_value_cota
    global preview_cnpj

    # Tratar o caso de loop de CNPJ
    current_cnpj = row['CNPJ_FUNDO_CLASSE']
    if preview_cnpj != current_cnpj:
        started_value_cota = 0
    preview_cnpj = current_cnpj

    investment_date_start = json_data['INVESTMENTS'][row['CNPJ_FUNDO_CLASSE']]['HistoricoCompraDatas'][0]
    investment_date_row = row['DT_COMPTC'][2:]

    boolean_verification = is_first_date_previous_than_second(investment_date_start, investment_date_row)

    if boolean_verification == False:

        # Isso pode dar erro, estou seguindo a regra que um usuário só pode comprar quando cotado o valor, pois se não, como afirmar qual foi a data?
        if investment_date_start == investment_date_row:
            started_value_cota = float(row['VL_QUOTA'])
        return 0.00
    
    valuation_percentual = ((float(row['VL_QUOTA']) / started_value_cota) - 1) * 100
    return round(valuation_percentual, 2)

list_investments_filtered['VALUATION_PERCENT'] = list_investments_filtered.apply(calculate_valuation, axis=1)

In [ ]:
# Lista as colunas na ordem correta

list_investments_filtered = list_investments_filtered[["CNPJ_FUNDO_CLASSE", "NOME_FUNDOS", "DT_COMPTC", "VL_QUOTA", "VALUATION_PERCENT", "VL_TOTAL","VL_PATRIM_LIQ", "CAPTC_DIA", "RESG_DIA","NR_COTST"]].copy()

display(list_investments_filtered)

In [ ]:
# Armanzenar os dados formatados da tabela

cvm_data_dir = os.path.join(os.path.abspath('../archives/'), json_data['CONFIG']['DIR_CVM_DATA_NAME'])

if not os.path.exists(cvm_data_dir):
    os.mkdir(cvm_data_dir)

group_data_by_nome = list_investments_filtered.groupby('NOME_FUNDOS')
dataframe_by_nome_fundo = {}

for nome_fundo, data in group_data_by_nome:
    archive_name = nome_fundo.replace(' ', '_') + '.csv'
    dataframe_by_nome_fundo[nome_fundo] = data

    data.to_csv(os.path.join(cvm_data_dir, archive_name), index=False, sep=';')